In [3]:
import os
import pandas as pd
import geopandas as gpd
from arcgis import GIS
from mtcpy.agol import pull_geotable_agol
from dotenv import load_dotenv

load_dotenv()

True

In [5]:
gis = GIS(
    url="https://mtc.maps.arcgis.com/home",
    username=os.getenv("ARCGIS_USER"),
    password=os.getenv("ARCGIS_PASSWORD"),
)

In [6]:
epc_2050_gdf = pull_geotable_agol(
    base_url="https://services3.arcgis.com/i2dkYWmb4wHvYPda/arcgis/rest/services/communities_of_concern_2020_acs2018/FeatureServer/0",
    client=gis,
    reproject_to_analysis_crs=False,
)

Breaking feature service layer IDs into 8 chunks


In [7]:
epc_2050p_gdf = pull_geotable_agol(
    base_url="https://services3.arcgis.com/i2dkYWmb4wHvYPda/arcgis/rest/services/draft_equity_priority_communities_pba2050plus_acs2022a/FeatureServer/0",
    client=gis,
    reproject_to_analysis_crs=False,
)

Breaking feature service layer IDs into 8 chunks


In [11]:
epc_2050_gdf.rename(columns={"geoid": "tract_geoid"}, inplace=True)

In [20]:
epc_merged_df = pd.merge(
    epc_2050_gdf[["tract_geoid", "tot_pop", "epc_2050"]],
    epc_2050p_gdf[["tract_geoid", "tot_pop_poc", "epc_2050p"]],
    on="tract_geoid",
    how="outer",
    indicator=True,
)

In [28]:
# get 2022 regional population
epc_2050p_gdf['tot_pop_poc'].sum()

7685888

In [24]:
epc_only = epc_merged_df.query("epc_2050 == 1 or epc_2050p == 1")
epc_only.groupby("_merge", observed=False).aggregate(
    {"tot_pop": "sum", "tot_pop_poc": "sum"}
)

,tot_pop,tot_pop_poc
_merge,,
left_only,308201.0,0.0
right_only,0.0,264229.0
both,1565218.0,1561909.0


In [27]:
pop_2050 = epc_only.query("_merge == 'left_only'")["tot_pop"].sum()
pop_2050p = epc_only.query("_merge.isin(['right_only', 'both'])")["tot_pop_poc"].sum()

total_pop = pop_2050 + pop_2050p
total_pop

2134339.0